# Learning the Basics of QisKit

### Making Quantum Circuits

In [2]:
from qiskit import QuantumCircuit

# Create a quantum circuit containing a single qubit
qc = QuantumCircuit(1)

# Add a x gate to the single qubit
qc.x(0)

# Draw the circuit in ascii(most efficent)
qc.draw("text")

┌───┐
q: ┤ X ├
   └───┘

### Arbitrary Unitary
In this case I make an arbitrary unitary which results in a rotation of a state about an axis $n$, by angle $\theta$, with phase factor $e^{i\alpha}$.

Notes:\
Qubits are initilized to $\ket{0}$.\
Unitaries on single qubits can be realized as rotations about an axis on a state on the bloch sphere.

In [3]:
import numpy as np

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import *
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector

# An arbitrary single qubit unitary operator can be written in the form U = exp(ia)R_n(o).

def U (n, o, a): # n: Rotation axis; o: Angle of rotation; a: Phase.

    return np.exp(a*1j)*Rn(n, o)

def Rn (n, o): # n: Rotation axis; o: Angle of rotation.

    # Identity matrix
    I = np.array([[1,0],
                  [0,1]])
    
    # X-Pauli matrix
    X = np.array([[0,1],
                  [1,0]])
    
    # Y-Pauli matrix
    Y = np.array([[0,-1j],
                  [1j,0]])
    
    # Z-Pauli matrix
    Z = np.array([[1,0],
                  [0,-1]])

    return (np.cos(o/2)*I)-(np.sin(o/2)*(n[0]*X + n[1]*Y + n[2]*Z))*1j

n = np.array([1, 0, 0]) # Rotation axis
o = np.pi # Angle of rotation
a = np.pi # Phase

# Instantiate quantum and classical registers each of 1 bit.
qr = QuantumRegister (1, 'q')
cr = ClassicalRegister(1, 'measurment')

# Instantiate a quantum circuit of 1 qubit and 1 cbit.
qc = QuantumCircuit(qr, cr)
qc.append (                                 # Add a new gate to the circuit.
    UnitaryGate(U(n, np.pi/2, 0), "My Gate"), # Gate to be added.
    [0]                                     # Qubit(s) on which it is acting.
)
qc.measure(0,0)

# Draw the quantum circuit.
qc.draw(output = "text")

┌─────────┐┌─┐
           q: ┤ My Gate ├┤M├
              └─────────┘└╥┘
measurment: 1/════════════╩═
                          0

See if I can plot the statevector of a single qubit

In [4]:
# Store the state vectors of the qubits and plot them on the Bloch sphere.
#psi = Statevector(qc.qubits)
#plot_bloch_multivector(psi)

### Measurments?

In [5]:
from qiskit.primitives import StatevectorSampler
 
sampler = StatevectorSampler(default_shots = 1) # Record a single sample of the circuit.
result = sampler.run([qc]).result()[0] # Run the sampler on the quantum circuit and store the result.

data = result.data # Get the data from the result of running the circuit.


# String to class parameter???
counts = data.measurment.get_counts()
print(f"The counts are: {counts}")


The counts are: {'0': 1}


### Performing Full State Tomography with Maximum Likelihood Estimation
I chose Maximum Likelihood Estimation as it gaurentees a valid density matrix (need to check why) unlike other methods like linear inversion.\
Sudo-code from: https://wiki.veriqloud.fr/index.php?title=Full_Quantum_state_tomography_with_Maximum_Likelihood_Estimation 

In [6]:
import numpy as np

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import *
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler

# Pick a measurment basis. For a single qubit pure state it is sufficient
# to measure in the x, y, and z basis.

# Measurments in qiskit are taken in the computational basis.
# Therefore to measure in a basis other than the z we must rotate the state
# to project the x and y components into the computational basis.

# I will thus represent my basis as circuits which rotate the state so
# measurents in the z/computational basis corrospond to a measurment in the 
# target basis prior to rotation.

sampler = StatevectorSampler(default_shots = 1) # Instatiate a sampler that samples a single shot of the circuit.

xMeasure = QuantumCircuit(1,1)
xMeasure.append(HGate(), [0])
xMeasure.measure(0,0)

yMeasure = QuantumCircuit(1,1)
yMeasure.append(SdgGate(), [0])
yMeasure.append(HGate(), [0])
yMeasure.measure(0,0)

zMeasure = QuantumCircuit(1,1)
zMeasure.measure(0,0)

basis = [xMeasure, yMeasure, zMeasure]

# Circuit preparing a single qubit state. (Just apply the hadamard for now)
qc = QuantumCircuit(1,1)
qc.append(HGate(), [0])


Implementation of the algorithm.

In [7]:
n = 100 # Number of samples (single shots)

for i in basis:
    mj = 0
    for j in range(n):
        

        result = sampler.run([qc.compose(i)]).result()[0] # Run the sampler on the quantum circuit and store the result.
        data = result.data
        counts = data.c.get_counts()
        if "1" in counts:
            mij = -1 
        else:
            mij = 1
        
        mj += mij/n
    print(mj)

1.0000000000000007
-0.20000000000000004
0.020000000000000004


In [ ]:
from scipy import optimize

def lhf(ts): # Likelihood function
    for i in ts:
        